1. Upload [Full TMDB Movies Dataset 2024 (1M Movies) Kaggle Dataset](https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies) to Google Drive root folder.

2. Mount the Google Drive in Colab

3. Run the Script

In [1]:
!pip install -U polars[gpu] --extra-index-url=https://pypi.nvidia.com
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu11
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
# !pip install hvplot jupyter_bokeh holoviews==1.19 pynvml

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Using cached cuda_python-11.8.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached cuda_python-11.8.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.7 MB)
  Attempting uninstall: cuda-python
    Found existing installation: cuda-python 12.6.0
    Uninstalling cuda-python-12.6.0:
      Successfully uninstalled cuda-python-12.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.8.3 requires cuda-python<13.0a0,>=12.0, but you have cuda-python 11.8.3 which is incompatible.
rmm-cu12 24.8.2 requires cuda-python<13.0a0,>=12.0, but you have cuda-python 11.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Using cached cu

In [2]:
import polars as pl

df = pl.scan_csv('/content/drive/MyDrive/TMDB_movie_dataset_v11.csv')

print(df)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

Csv SCAN [/content/drive/MyDrive/TMDB_movie_dataset_v11.csv]
PROJECT */24 COLUMNS


In [3]:
%%time

# Group by production companies and calculate the average revenue
max_revenue_per_company = (df.group_by('production_companies')
                                    .agg(pl.col('revenue').max())
                                    .sort("revenue", descending=True)
                                    .head()
                                    .collect())

CPU times: user 1.24 s, sys: 347 ms, total: 1.58 s
Wall time: 1.48 s


In [4]:
gpu_engine = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=True, # Fail loudly if we can't run on the GPU.
)

In [6]:
%%time

# Group by production company and calculate the average revenue
max_revenue_per_company = (df.group_by('production_companies')
                                    .agg(pl.col('revenue').max())
                                    .sort("revenue", descending=True)
                                    .head()
                                    .collect(engine=gpu_engine))

CPU times: user 153 ms, sys: 324 ms, total: 477 ms
Wall time: 772 ms
